In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo

# Fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# Data (as pandas dataframes)
x = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

# Assuming 'y' is continuous (regression problem)

# Encode 'Diagnosis' column to numerical values
y['Diagnosis'] = y['Diagnosis'].map({'M': 1, 'B': 0})

# Create a continuous DataFrame
y_continuous = pd.DataFrame(y, columns=['Diagnosis'])

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y_continuous, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert data to PyTorch tensors
x_train_tensor = torch.FloatTensor(x_train_scaled)
y_train_tensor = torch.FloatTensor(np.array(y_train['Diagnosis']))
x_test_tensor = torch.FloatTensor(x_test_scaled)
y_test_tensor = torch.FloatTensor(np.array(y_test['Diagnosis']))
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


# Combine input features and target labels into a TensorDataset
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# Set random seed for reproducibility
# torch.manual_seed(42)

/tmp/ipykernel_6722/316924810.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Diagnosis'] = y['Diagnosis'].map({'M': 1, 'B': 0})


In [26]:


# Define the neural network model for regression
class RegressionNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(RegressionNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


def runModel(bs, layer1, layer2, lr):
    # Instantiate the regression model
    input_size = x_train_tensor.shape[1]
    hidden_size1 = layer1
    hidden_size2 = layer1
    output_size = 1  # Single output node for regression

    regression_model = RegressionNeuralNetwork(input_size, hidden_size1, hidden_size2, output_size)

    # Define loss function and optimizer for regression
    criterion = nn.MSELoss()
    optimizer = optim.Adam(regression_model.parameters(), lr=lr)

    # Training the regression model
    num_epochs = 10
    batch_size = bs

    for epoch in range(num_epochs):
        regression_model.train()
        for i in range(0, len(x_train_tensor), batch_size):
            inputs = x_train_tensor[i:i+batch_size]
            labels = y_train_tensor[i:i+batch_size]

            optimizer.zero_grad()
            outputs = regression_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluation on the test set
    regression_model.eval()
    with torch.no_grad():
        test_outputs = regression_model(x_test_tensor)
        mse_loss = criterion(test_outputs, y_test_tensor)
        return mse_loss.item()


In [27]:
# experiment 1: batch size
i = 4
while i < 513:
    loss = 0
    for j in range(500):
        loss += runModel(i, 64, 32, 0.001)
    loss = loss / 500
    print("Batch size of ", i, " with a loss of ", loss)
    i *= 2

# experiment 2: first layer size
i = 4
while i < 513:
    loss = 0
    for j in range(500):
        loss += runModel(32, i, 32, 0.001)
    loss = loss / 500
    print("First layer size of ", i, " with a loss of ", loss)
    i *= 2

# experiment 3: second layer size
i = 4
while i < 513:
    loss = 0
    for j in range(500):
        loss += runModel(32, 64, i, 0.001)
    loss = loss / 500
    print("Second layer size of ", i, " with a loss of ", loss)
    i *= 2

# experiment 4: learning rate
i = 0.000001
while i < 1.1:
    loss = 0
    for j in range(500):
        loss += runModel(32, 64, 32, i)
    loss = loss / 500
    print("Learning rate of ", i, " with a loss of ", loss)
    i *= 10

Batch size of  4  with a loss of  0.029195287764072417
Batch size of  8  with a loss of  0.02952264230325818
Batch size of  16  with a loss of  0.03243524648249149
Batch size of  32  with a loss of  0.04014174184203148
Batch size of  64  with a loss of  0.0529320468455553
Batch size of  128  with a loss of  0.07105891964584589
Batch size of  256  with a loss of  0.09721524353325367
Batch size of  512  with a loss of  0.1316498640924692
First layer size of  4  with a loss of  0.15551776701956987
First layer size of  8  with a loss of  0.07899072889983653
First layer size of  16  with a loss of  0.060660927325487134
First layer size of  32  with a loss of  0.04917455017194152
First layer size of  64  with a loss of  0.0400213914103806
First layer size of  128  with a loss of  0.036586372021585704
First layer size of  256  with a loss of  0.03741926873475313
First layer size of  512  with a loss of  0.044506886050105096
Second layer size of  4  with a loss of  0.04040754563733935
Second l